IMPORTS

In [429]:
import pandas as pd
import datetime as dt
import numpy as np
import ast
import re

FONCTIONS

In [430]:
apparition_realisateur = 35
apparition_acteur = 75

In [431]:
colonnes_limite = {'production_companies_name' : [3, 275],
                    'title_debut' : [6, 3]}

In [432]:
def name(liste_films, df):

    df = df.drop(['ordering', 'category', 'job', 'characters'], axis = 1)
    df['nconst'] = df['nconst'] + ', '
    df = df.groupby('tconst').sum()
    df = df.reset_index()

    df = pd.merge(liste_films,
                  df,
                  left_on = 'film_id',
                  right_on = 'tconst',
                  how = 'left')

    df = df.drop('film_id', axis = 1)
    df['nconst'] = df['nconst'].astype(str)
    df['nconst'] = df['nconst'].apply(lambda x : x.split(','))
    df = df.explode('nconst')
    df['tconst'] = df['tconst'] + ','
    df = df.groupby('nconst').sum()
    df = df.reset_index()
    df = df[~(df['nconst']== ' ')]
    df['nconst'] = df['nconst'].astype(str)
    df['tconst'] = df['tconst'].astype(str)
    df['nconst'] = df['nconst'].apply(lambda x : x.replace(' ', ''))
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', ''))

    return df

In [433]:
def acteur_name(df):

    df['tconst_y'] = df['tconst_y'].astype(str)
    df['tconst'] = df['tconst_x'] + df['tconst_y']
    df = df[['nconst', 'primaryName', 'tconst']]
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", ''))
    df['tconst'] = df['tconst'].apply(lambda x : x.split(','))
    df = df.rename({'tconst': 'knownForTitles'}, axis = 1)
    df = df.rename({'nconst': 'acteur'}, axis = 1)

    return df

In [434]:
def realisateur_name(df):
    
    df = df.groupby('nconst').sum()
    df = df.reset_index()
    df = pd.merge(df,
                  df_title_crew,
                  on = 'nconst',
                  how = 'left')

    df = df[~(df['nconst'] == 'nan')]
    df['tconst_x'] = df['tconst_x'].astype(str)
    df['tconst_y'] = df['tconst_y'].astype(str)
    df['tconst'] = df['tconst_x'] + ',' + df['tconst_y']
    df = df.drop(['tconst_y', 'tconst_x'], axis = 1)
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", '').replace(",,", ','))
    df['tconst'] = df['tconst'].astype(str)
    df['tconst'] = df['tconst'].apply(lambda x : x.split(','))
    df['tconst'] = df['tconst'].apply(lambda x : set(x))
    df['tconst'] = df['tconst'].apply(lambda x : list(x))

    df = pd.merge(df,
                  df_name_basics,
                  on = 'nconst',
                  how = 'left')

    df['tconst_x'] = df['tconst_x'].astype(str)
    df['tconst_y'] = df['tconst_y'].astype(str)
    df['tconst'] = df['tconst_x'] + ',' + df['tconst_y']
    df = df.drop(['tconst_y', 'tconst_x'], axis = 1)
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", '').replace(",,", ','))
    df['tconst'] = df['tconst'].astype(str)
    df['tconst'] = df['tconst'].apply(lambda x : x.split(','))
    df['tconst'] = df['tconst'].apply(lambda x : set(x))
    df['tconst'] = df['tconst'].apply(lambda x : list(x))

    df = df.rename({'tconst': 'knownForTitles'}, axis = 1)
    df = df.rename({'nconst': 'realisateurs'}, axis = 1)
    df = df.explode('knownForTitles')
    df = df[~(df['knownForTitles'] == '')]

    return df

In [435]:
def transfo_liste(x):
  if isinstance(x, str):
    return ast.literal_eval(x)
  else:
    return x

In [436]:
def debut(titre):
    titre = titre.lower().replace(' ', '')
    titre_final = ''
    for n in range(len(titre)):
        if titre[n].isalpha():
            titre_final += titre[n]
    return titre_final[0:20]

In [437]:
def bool_colonnes(df_films, colonne):

    def len_pandas(x):
        x_escaped = re.escape(x)
        return df_films[df_films[f'{colonne}2'].fillna('').str.contains(x_escaped, na=False, regex=True)].shape[0]

    def debut2(titre):
        titre = titre.lower().replace(' ', '')
        titre_final = ''
        for n in range(len(titre)):
            if titre[n].isalpha() or titre[n].isnumeric():
                titre_final += titre[n]
            else:
                break
        if len(titre_final) > 20:
            titre_final = titre_final[0:20]
        return titre_final

    if colonne in ['acteur', 'realisateurs', 'title_2', 'production_companies_name', 'genre']:
        df_films[colonne] = df_films[colonne].astype(str)

    if type(df_films[colonne].iloc[0]) == str:
        if colonne == 'title_debut':
            df_films[f'{colonne}2'] = df_films['title'].apply(lambda x: x.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(" ", '').lower())
            df_films[f'{colonne}'] = df_films[f'{colonne}2'].apply(debut2) 
            df_films[f'{colonne}'] = df_films[f'{colonne}'].apply(lambda x : x.split(' '))
        else:
            df_films[f'{colonne}2'] = df_films[colonne].apply(lambda x: x.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(" ", '').lower())
            df_films[f'{colonne}'] = df_films[f'{colonne}2'].apply(lambda x : x.split(','))


    tous_les_genres = set()


    for genres in df_films[colonne] :
        tous_les_genres.update(genres)
    tous_les_genres = list(tous_les_genres)

    if "" in tous_les_genres:
        tous_les_genres.remove("")

    if colonne in colonnes_limite.keys():

        tous_les_genres2 = []
        for element in tous_les_genres:
            if len_pandas(element) >= colonnes_limite[colonne][1]:
                tous_les_genres2.append(element)

        last_list = []
        for element in tous_les_genres2:
            if len(element) >= colonnes_limite[colonne][0]:
                last_list.append(element)

        tous_les_genres = last_list

    if colonne == 'production_companies_name':
        df_films['production_companies_name_Marvel'] = df_films['production_companies_name2'].apply(lambda x: 'marvel' in x)
        df_films['production_companies_name_Disney'] = df_films['production_companies_name2'].apply(lambda x: 'disney' in x)
        
    for genre in tous_les_genres:
        df_films[f'{colonne}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
 
    # df_films = df_films.drop(f'{colonne}2', axis = 1)
    df_films = df_films.drop(colonne, axis = 1)

In [438]:
def transfo_col_bool(df_films):

    df_films_not_bool = df_films.select_dtypes(exclude = 'bool')
    df_films_bool = df_films.select_dtypes(include = 'bool')
    df_films_bool = df_films_bool.astype(str)
    df_films_bool = df_films_bool.replace('True', '1').replace('False', '0')
    df_films_bool = df_films_bool.astype(int)
    df_films = pd.concat([df_films_not_bool, df_films_bool], axis = 1)
    df_films_not_bool, df_films_bool = 0, 0

    return df_films

BASES ET MERGES

In [439]:
df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/3595914245.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])


In [440]:
df_title_ratings = pd.read_csv('BD/title.ratings.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])

In [441]:
df1 = pd.merge(df_title_basics,
               df_title_ratings,
               how = 'outer',
               on = 'tconst')

In [442]:
df_title_ratings, df_title_basics = 0, 0

In [443]:
df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/2474832937.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])


In [444]:
df_3 = pd.merge(df1,
                df_title_akas,
                how = 'outer',
                left_on = 'tconst',
                right_on = 'titleId')

In [445]:
# ON GARDE QUE LES FILMS

df_3 = df_3[(df_3['titleType'] == 'movie')]

In [446]:
df1, df_title_akas = 0, 0

In [447]:
df_tmbd_full = pd.read_csv('BD/tmdb_full.csv', na_values= ['\\N'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/2845617658.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd_full = pd.read_csv('BD/tmdb_full.csv', na_values= ['\\N'])


In [448]:
df_films = pd.merge(df_tmbd_full,
                df_3,
                how = 'outer',
                left_on = 'imdb_id',
                right_on = 'tconst')

In [449]:
df_tmbd_full, df_3 = 0, 0

COLONNES NUMERIQUES

In [450]:
# On remplit les cellules vides par 0

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].fillna(0)

In [451]:
# On remplace les '\\N' par 0

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].replace('\\N', 0)

In [452]:
# On met ces colonnes au format numérique

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].apply(pd.to_numeric)

In [453]:
# On transforme les types de données pour 'release_date'

df_films['release_date'] = pd.to_datetime(df_films['release_date'])
df_films['release_date'] = df_films['release_date'].replace('\\N', 0)
df_films['release_date'] = df_films['release_date'].dt.year

In [454]:
# On transforme les types de données pour 'startYear'

df_films['startYear'] = df_films['startYear'].replace('\\N', 0)
df_films['startYear'] = pd.to_numeric(df_films['startYear'], downcast="integer")

In [455]:
# On crée la colonne year_exact

df_films['year_exact'] = df_films[['release_date', 'startYear']].max(axis = 1)

In [456]:
# On crée la colonne 'Decennie'

df_films['Decennie'] = ''
df_films['year_exact'] = df_films['year_exact'].fillna('0')
df_films['year_exact'] = df_films['year_exact'].replace('nan', '0')
df_films['year_exact'] = df_films['year_exact'].astype(str)

In [457]:
df_films['Decennie'] = df_films['year_exact'].str[:3] + "0"
df_films['year_exact'] = pd.to_numeric(df_films['year_exact'])
df_films['Decennie'] = pd.to_numeric(df_films['Decennie'])

RUNTIME

In [458]:
# On met de côté les valeurs nulles

df_null1 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_null2 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'] == 0))]
df_null3 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'].isna()))]
df_null4 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'] == 0))]

In [459]:
df_null = pd.concat([df_null1, df_null2, df_null3, df_null4])

In [460]:
df_null1, df_null2, df_null3, df_null4 = 0, 0, 0, 0

In [461]:
# On prend les valeurs utiles

df_utile = df_films[~((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'] == 0))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'].isna()) & (df_utile['runtime'] == 0))]

In [462]:
# Calcul différence de runtime des 2 tables

df_utile['difference_runtime'] = abs(df_utile['runtime'] - df_utile['runtimeMinutes'])

In [463]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_utile[(df_utile['runtimeMinutes'] == df_utile['runtime'])]

In [464]:
# Création colonne RuntimeExact
df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']

df_a_travailler_1 = df_utile[~(df_utile['runtimeMinutes'] == df_utile['runtime'])]

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/3570380868.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']


In [465]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]

In [466]:
# On conserve la donnée existante

df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/3521993277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)


In [467]:
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]
df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/1261991755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])


In [468]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_runtime'] <= 20]

In [469]:
# On prend comme base la valeur max des 2 colonnes

df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/880983650.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [470]:
df_a_travailler_3 = df_a_travailler_2[~(df_a_travailler_2['difference_runtime'] <= 20)]

In [471]:
# Pour le restant des données, on conserve la valeur max des 2 colonnes

df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/264539281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [472]:
df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])

In [473]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [474]:
df_films = pd.concat([df_final, df_null])

In [475]:
df_final, df_null = 0, 0

VOTES

In [476]:
# On met de côté les valeurs inutiles

df_null5 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_null6 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'] == 0))]
df_null7 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'].isna()))]
df_null8 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'] == 0))]

In [477]:
df_null9 = pd.concat([df_null5, df_null6, df_null7, df_null8])

In [478]:
df_null5, df_null6, df_null7, df_null8 = 0, 0, 0, 0

In [479]:
# On prend les valeurs utiles

df_ratings = df_films[~((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'] == 0))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'].isna()) & (df_ratings['averageRating'] == 0))]

In [480]:
df_ratings['vote_average'] = round(df_ratings['vote_average'], 1)

In [481]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_ratings[df_ratings['averageRating'] == df_ratings['vote_average']]
df_a_travailler_1 = df_ratings[~(df_ratings['averageRating'] == df_ratings['vote_average'])]

In [482]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[(df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0)]
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0))]

In [483]:
df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/2546417221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])


In [484]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] <= 1]
df_a_travailler_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] > 1]

In [485]:
df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
df_a_travailler_3['vote_exact'] = round(df_a_travailler_3['vote_exact'], 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/4062215657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/4062215657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travaill

In [486]:
# On crée la colonne vote_exact pour garder la moyenne des votes

df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_3['vote_exact'] = df_a_garder_3[['averageRating', 'vote_average']].mean(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/3653439724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/3653439724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/36534397

In [487]:
# On concatène tous les tableaux conservés

df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])
df_final['arrondi_vote_exact'] = round(df_final['vote_exact'], 0)

In [488]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [489]:
df_films = pd.concat([df_final, df_null9])

In [490]:
df_final, df_null9 = 0, 0

GENRE

In [491]:
df_films = df_films.rename(columns = {'genres_x' : 'genres_y', 'genres_y' : 'genres_x'})

In [492]:
df_films[df_films[['genres_y', 'genres_x']] == '\\N'] = df_films[df_films[['genres_y', 'genres_x']] == '\\N'].replace('\\N', np.nan)

In [493]:
df_films['genres_y'] = df_films['genres_y'].apply(transfo_liste)

In [494]:
df_films['genres_y'] = df_films['genres_y'].apply(lambda x: x if (type(x) == list) else [])

In [495]:
df_films['genres_x'] = df_films['genres_x'].str.replace(",", ", ")

In [496]:
df_films['genres_x_list'] = df_films['genres_x'].fillna('').str.split(', ')

In [497]:
df_films['genre'] = df_films.apply(lambda row: list(set(row['genres_y'] + row['genres_x_list'])),axis=1)

In [498]:
df_films['genre'] = df_films['genre'].apply(lambda x: x if x != [] else np.nan)

TITLE

In [499]:
df_films['title'] = df_films['title_y'].fillna(df_films['title_x']).fillna(df_films['primaryTitle']).fillna(df_films['original_title']).fillna(df_films['originalTitle'])

ADULT

In [500]:
df_films['adult'] = (df_films['isAdult'] == 1) | df_films['adult']

TCONST

In [501]:
df_films['film_id'] = df_films['imdb_id'].fillna(df_films['tconst']).fillna(df_films['titleId'])

VOTE COUNT

In [502]:
df_films['vote_count_mean'] = (df_films['vote_count'] + df_films['numVotes'])/2

FINAL LANGUAGE

In [503]:
colonnes_y = ['production_countries', 'spoken_languages', 'production_companies_country']

for cl in colonnes_y:
 
    df_films[cl] = df_films[cl].apply(transfo_liste)
    df_films[cl] = df_films[cl].apply(lambda x: x if (type(x) == list) else [])

In [504]:
df_films['original_language_x_list'] = df_films['original_language'].fillna('').str.split(', ')

In [505]:
df_films['language_x_list'] = df_films['language'].fillna('').str.split(', ')

In [506]:
df_films['final_language'] = df_films.apply(lambda row: list(set(row['production_countries'] + row['spoken_languages'] + row['production_companies_country'] + row['original_language_x_list'] + row['language_x_list'])),axis=1)

In [507]:
df_films['final_language'] = df_films['final_language'].apply(lambda x: x if x != [] else np.nan)

In [508]:
df_films['final_language'] = df_films['final_language'].apply(lambda x: ', '.join(x) if type(x) == list else x)

In [509]:
df_films = df_films[(df_films['final_language'].isna() == True) | (df_films['final_language'] == '') | (df_films['final_language'].str.contains('fr') == True) | (df_films['final_language'].str.contains('FR') == True)  | (df_films['final_language'].str.contains('us') == True) | (df_films['final_language'].str.contains('US') == True)]

CLEAN FINAL DU MERGE ET GROUP BY

In [510]:
df_str = df_films.select_dtypes(exclude = 'number')

In [511]:
df_str['language_fr'] = df_str['final_language'].apply(lambda x : 1 if ('fr' in x or 'FR' in x) else 0)

In [512]:
df_str = df_str.sort_values(by = 'language_fr', ascending = False)

In [513]:
df_str_drop_dup = df_str.drop_duplicates(subset = ['film_id'], keep = 'first')

In [514]:
df_num = pd.concat([df_films['film_id'], df_films.select_dtypes(include = 'number')], axis = 1)

In [515]:
df_num = df_num.drop(['revenue', 'budget'], axis = 1)

In [516]:
df_revenue_budget = df_films[['film_id', 'revenue', 'budget']]

In [517]:
df_revenue_budget_gb = df_revenue_budget.groupby('film_id').max()

In [518]:
df_num_gb = df_num.groupby('film_id').max()

In [519]:
df_num2 = pd.merge(df_num_gb,
                   df_revenue_budget_gb,
                   how = 'outer',
                   on = 'film_id')

In [520]:
df_films = pd.merge(df_str_drop_dup,
                   df_num2,
                   on = 'film_id',
                   how = 'left')

In [521]:
df_num2, df_str_drop_dup, df_num_gb, df_revenue_budget_gb, df_revenue_budget, df_num, df_str = 0, 0, 0, 0, 0, 0, 0

In [522]:
df_films = df_films[df_films['status'] == 'Released']

In [523]:
df_films = df_films[~(df_films['vote_count_mean'] == 0)] # A voir si on le conserve
df_films = df_films[~(df_films['year_exact'] == 0)]
df_films = df_films[~(df_films['runtime_exact'].isna() == True)]
df_films['region'] = df_films['region'].fillna('Unknown')

CHANGEMENT POUR LANGUE FR

In [524]:
df_films['prod_FR'] = df_films.apply(lambda row : 1 if 'FR' in row['production_companies_country'] else 0, axis = 1)

CHANGEMENT POUR FILM US

In [525]:
df_films['prod_US'] = df_films.apply(lambda row : 1 if 'US' in row['production_companies_country'] else 0, axis = 1)

In [526]:
df1, df_3, df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4, df_a_travailler_1, df_a_travailler_2, df_a_travailler_3, df_final, df_null = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [527]:
df_films[df_films.select_dtypes('number') == True] = df_films[df_films.select_dtypes('number') == True].fillna(0)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/3129310626.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_films[df_films.select_dtypes('number') == True] = df_films[df_films.select_dtypes('number') == True].fillna(0)


In [528]:
df_films[['vote_exact', 'arrondi_vote_exact']] = df_films[['vote_exact', 'arrondi_vote_exact']].astype(float)

In [529]:
df_films['titleType'] = df_films['titleType'].fillna('movie')

In [530]:
df_num = df_films.select_dtypes(include = 'number')
df_num = df_num.fillna(0)
df_str = df_films.select_dtypes(exclude = 'number')
df_str = df_str.fillna("Unknown")

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/3351577065.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_str = df_str.fillna("Unknown")


In [531]:
df_films = pd.concat([df_num, df_str], axis = 1)

In [532]:
df_str, df_num = 0, 0

In [533]:
df_films = df_films[(df_films['final_language'].str.contains('fr') == True) | (df_films['final_language'].str.contains('FR') == True)]

AJOUT TMDB FILMS

In [534]:
df_tmbd_full = pd.read_csv('BD/tmdb_full.csv', na_values= ['\\N'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/2845617658.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd_full = pd.read_csv('BD/tmdb_full.csv', na_values= ['\\N'])


In [535]:
for element in df_tmbd_full.columns:
    df_tmbd_full = df_tmbd_full.rename(columns = {element : f'{element}_tmdb'})

for element in df_films.columns:
    df_films = df_films.rename(columns = {element : f'{element}_df'})

In [576]:
df_essai_tmdb = pd.merge(df_films,
                         df_tmbd_full,
                         how = 'outer',
                         right_on = 'imdb_id_tmdb',
                         left_on = 'film_id_df')

In [577]:
df_essai_tmdb = df_essai_tmdb.drop(['language_fr_df', 'genres_y_df', 'genres_x_df', 'genres_x_list_df', 'original_title_df', 'title_x_df', 'titleType_df', 'titleId_df', 'title_y_df', 'original_title_tmdb', 'runtime_df', 'runtimeMinutes_df', 'difference_runtime_df', 'vote_average_df', 'difference_vote_df', 'vote_count_mean_df', 'vote_count_df', 'vote_count_mean_df'], axis = 1)

In [578]:
df_essai_tmdb['film_id_final'] = df_essai_tmdb['film_id_df'].fillna(df_essai_tmdb['imdb_id_tmdb'])
df_essai_tmdb = df_essai_tmdb.drop(['film_id_df', 'imdb_id_tmdb'], axis = 1)

In [579]:
df = pd.read_csv('BD/P2_G5_films.csv.gz', compression = 'gzip')
colonnes_df = list(df.columns)
colonnes_df_2 = []

for element in colonnes_df:
    if '_out_KNN' in element:
        colonnes_df_2.append(element[:-8])
    else:
        colonnes_df_2.append(element)

colonnes_df_2 = colonnes_df_2[0:24]

In [580]:
df_essai_tmdb = df_essai_tmdb.rename({'release_date_tmdb' : 'year_tmdb',
                                      'vote_average_tmdb' : 'vote_exact_tmdb'}, axis = 1)
df_essai_tmdb['vote_exact_tmdb'] = df_essai_tmdb['vote_exact_tmdb'].astype(float)

df_essai_tmdb['vote_exact_int_tmdb'] = ''
df_essai_tmdb['vote_exact_int_tmdb'] = df_essai_tmdb['vote_exact_tmdb'].apply(lambda x : round(x, 0))
df_essai_tmdb['arrondi_vote_exact'] = df_essai_tmdb['arrondi_vote_exact_df'].fillna(df_essai_tmdb['vote_exact_int_tmdb'])

df_essai_tmdb['arrondi_vote_exact'] = df_essai_tmdb['arrondi_vote_exact'].astype(int)

In [581]:
df_essai_tmdb['year_tmdb'] = pd.to_datetime(df_essai_tmdb['year_tmdb'])
df_essai_tmdb['year_tmdb'] = df_essai_tmdb['year_tmdb'].apply(lambda x : x.year)
df_essai_tmdb['year_tmdb'] = df_essai_tmdb['year_tmdb'].fillna(0)
df_essai_tmdb['year_tmdb'] = df_essai_tmdb['year_tmdb'].astype(int)

df_essai_tmdb['Decennie_tmdb'] = ''
df_essai_tmdb['year_tmdb'] = df_essai_tmdb['year_tmdb'].fillna('0')
df_essai_tmdb['year_tmdb'] = df_essai_tmdb['year_tmdb'].replace('nan', '0')
df_essai_tmdb['year_tmdb'] = df_essai_tmdb['year_tmdb'].astype(str)
df_essai_tmdb['Decennie_tmdb'] = df_essai_tmdb['year_tmdb'].str[:3] + "0"
df_essai_tmdb['year_tmdb'] = pd.to_numeric(df_essai_tmdb['year_tmdb'])
df_essai_tmdb['Decennie_tmdb'] = pd.to_numeric(df_essai_tmdb['Decennie_tmdb'])

In [582]:
for element2 in colonnes_df_2:
    for element in df_essai_tmdb.columns:
        if element2 in element and 'int' not in element and 'arrondi' not in element and 'count' not in element :
            if 'df' in element:
                df_essai_tmdb = df_essai_tmdb.rename({element : f'{element2}_df'}, axis = 1)
            else:
                df_essai_tmdb = df_essai_tmdb.rename({element : f'{element2}_tmdb'}, axis = 1)

In [583]:
for element in colonnes_df_2:
    for element2 in df_essai_tmdb.columns:
        if element in element2:
            df_essai_tmdb[element2] = df_essai_tmdb[element2].astype(str)
            df_essai_tmdb[element2] = df_essai_tmdb[element2].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", '').replace(",,", ',').replace('nan', ''))
            df_essai_tmdb[element2] = df_essai_tmdb[element2].apply(lambda x : np.nan if x == '' else x)

In [584]:
for element2 in colonnes_df_2:
    for element in df_essai_tmdb.columns:
        if element2 in element :
            if f'{element2}_df' in df_essai_tmdb.columns and f'{element2}_tmdb' in df_essai_tmdb.columns:
                df_essai_tmdb[f'{element2}_final'] = df_essai_tmdb[f'{element2}_df'].fillna(df_essai_tmdb[f'{element2}_tmdb'])
                df_essai_tmdb = df_essai_tmdb.drop(f'{element2}_df', axis = 1)
                df_essai_tmdb = df_essai_tmdb.drop(f'{element2}_tmdb', axis = 1)
            elif f'{element2}_df' in df_essai_tmdb.columns and f'{element2}_tmdb' not in df_essai_tmdb.columns:
                df_essai_tmdb[f'{element2}_final'] = df_essai_tmdb[f'{element2}_df']
                df_essai_tmdb = df_essai_tmdb.drop(f'{element2}_df', axis = 1)
            elif f'{element2}_df' not in df_essai_tmdb.columns and f'{element2}_tmdb' in df_essai_tmdb.columns:
                df_essai_tmdb[f'{element2}_final'] = df_essai_tmdb[f'{element2}_tmdb']
                df_essai_tmdb = df_essai_tmdb.drop(f'{element2}_tmdb', axis = 1)

In [585]:
df_essai_tmdb = df_essai_tmdb.rename({'genre_df' : 'genre_final',
                                      'production_companies_name_df' : 'production_companies_name_final'}, axis = 1)

In [591]:
colonnes = []
for element in df_essai_tmdb.columns:
    if 'final' in element:
        colonnes.append(element[:-6])
        df_essai_tmdb = df_essai_tmdb.rename({element : element[:-6]}, axis =1)

df_essai_tmdb = df_essai_tmdb[colonnes]

,production_companies_name,genre,popularity,year_exact,Decennie,runtime_exact,vote_exact,arrondi_vote_exact,arrondi_vote_exact,prod_FR,...,overview,poster_path,production_countries,status,tagline,region,title,film_id,final_language,adult
0,NaN,NaN,3.754,NaN,1890,NaN,5.22,5,NaN,NaN,...,ThefirstwomantoappearinfrontofanEdisonmotionpi...,/sDbM3NnfnsEZYEJMt4O06Vb8Z9k.jpg,US,Released,NaN,NaN,Carmencita,tt0000001,NaN,False
1,[],"[Animation, ]",1.4,1892.0,1890.0,10.0,5.3,5,5.0,0.0,...,Shortfilmof300individuallypaintedimages.,/4ACC8V9ZSZ9AlWcAqUeF8aMcZV6.jpg,FR,Released,Unknown,Unknown,ClowdHisDogs,tt0000002,",FR,fr,xx",False
2,[],"[Comedy, , Animation]",4.353,1892.0,1890.0,4.0,6.0,6,6.0,0.0,...,"Onenight,ArlequincomestoseehisloverColombine.B...",/1e2ytD2cSxAz1r0SItxDmZUPb4i.jpg,FR,Released,Unknown,Unknown,PoorPierrot,tt0000003,",FR,fr",False
3,[],"[Animation, ]",1.129,1892.0,1890.0,15.0,4.4,4,4.0,0.0,...,Unknown,/osv0TcCymlsOucqkZBssSuS4tD.jpg,FR,Released,Unknown,Unknown,AGoodBeer,tt0000004,",FR,fr,xx",False
4,NaN,NaN,3.741,NaN,1890,NaN,5.556,6,NaN,NaN,...,Threemenhammeroanvilandpassabottleofbeeraround...,/c76bs0S90EFhB5ww3i6DlYQTVk.jpg,US,Released,NaN,NaN,BlacksmithScene,tt0000005,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309567,NaN,NaN,0.811,NaN,2010,NaN,7.1,7,NaN,NaN,...,NaN,/nn4MEcO0qYNgQIprIgn8OTEZib9.jpg,FR,Released,NaN,NaN,DanielleDarrieux:Ilestpolidêtregai!,tt9916192,NaN,False
309568,NaN,NaN,6.788,NaN,2020,NaN,6.388,6,NaN,NaN,...,"DjStephisayoungradiodeejayontherise,whogaineda...",/5pAAjVDpq4cCCsZsdivkCLMY6Hm.jpg,IT,Released,NaN,NaN,TheTalentoftheHornet,tt9916270,NaN,False
309569,"['INCAA', 'La Fidèle Production', 'Lamia Produ...","[Drama, History]",8.629,2020.0,2020.0,92.0,6.550000000000001,7,7.0,1.0,...,"FrenchBasqueCountry,year1609.Themenofasmallfis...",/lyB3yoQbGO3LiCVuSOD2g17Hr6p.jpg,"AR,ES,FR",Released,Unknown,AU,Coven,tt9916362,",es,AR,FR,eu,ES",False
309570,NaN,NaN,0.6,NaN,2010,NaN,0.0,0,NaN,NaN,...,OnedayApostolisdecidesthathewantstobeasuperher...,/zom5FHTqJjutluw4wHDsNxzCm8K.jpg,GR,Released,NaN,NaN,PinkTaxi,tt9916460,NaN,False


In [592]:
df_films = df_essai_tmdb

COLONNES ACTEUR ET REALISATEUR

In [593]:
liste_films = pd.DataFrame(df_films['film_id'])

In [594]:
liste_films = liste_films.drop_duplicates()

In [595]:
df_title_principals = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')

In [596]:
df_acteurs = df_title_principals[(df_title_principals['category'] == 'actor') | (df_title_principals['category'] == 'actress') | (df_title_principals['category'] == 'self')]

In [597]:
df_realisateurs = df_title_principals[df_title_principals['category'] == 'director']

In [598]:
df_title_principals = 0

In [599]:
df_title_crew = pd.read_csv('BD/title.crew.tsv.gz', compression = 'gzip', sep = '\t')

In [600]:
df_title_crew = df_title_crew.drop('writers', axis = 1)

In [601]:
df_title_crew = pd.merge(liste_films,
                         df_title_crew,
                         left_on = 'film_id',
                         right_on = 'tconst',
                         how = 'left')

In [602]:
df_title_crew = df_title_crew.drop('film_id', axis = 1)

In [603]:
df_title_crew = df_title_crew.rename({'directors': 'nconst'}, axis = 1)

In [604]:
df_title_crew['nconst'] = df_title_crew['nconst'].astype(str)

In [605]:
df_title_crew['nconst'] = df_title_crew['nconst'].apply(lambda x : x.split(","))

In [606]:
df_title_crew = df_title_crew.explode('nconst')

In [607]:
df_title_crew = df_title_crew[~(df_title_crew['nconst'] == " ")]

In [608]:
df_title_crew['tconst'] = df_title_crew['tconst'] + ', '

In [609]:
df_title_crew = df_title_crew.groupby('nconst').sum()

In [610]:
df_title_crew = df_title_crew.reset_index()

In [611]:
df_name_basics = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')

In [612]:
df_name_basics = df_name_basics.drop(['birthYear', 'deathYear', 'primaryProfession'], axis = 1)

In [613]:
df_name_basics = df_name_basics.rename({'knownForTitles' : 'tconst'}, axis = 1)

In [614]:
df_name_basics['tconst'] = df_name_basics['tconst'].apply(lambda x : x.split(','))

In [615]:
df_name_basics['nconst'] = df_name_basics['nconst'].astype(str)

In [616]:
df_name_basics['nconst'] = df_name_basics['nconst'].apply(lambda x : x.replace(' ', ''))

In [617]:
df_acteurs = name(liste_films, df_acteurs)

In [618]:
df_realisateurs = name(liste_films, df_realisateurs)

In [619]:
df_acteurs_name = pd.merge(df_acteurs,
                            df_name_basics,
                            on = 'nconst',
                            how = 'left')

In [620]:
df_acteurs = 0

In [621]:
df_acteurs_name = acteur_name(df_acteurs_name)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/3296483890.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", ''))
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/3296483890.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tconst'] = df['tconst'].apply(lambda x : x.split(','))


In [622]:
df_realisateurs_name = realisateur_name(df_realisateurs)

In [623]:
df_acteurs, df_realisateurs, df_name_basics, df_title_crew = 0, 0, 0, 0

In [624]:
df_realisateurs_name.to_csv('P2_G5_realisateurs.csv.gz', index=False, compression='gzip')
df_realisateurs = df_realisateurs_name

In [625]:
df_acteurs_name.to_csv('P2_G5_acteurs.csv.gz', index=False, compression='gzip')
df_acteurs = df_acteurs_name

In [626]:
# df_acteurs = pd.read_csv('BD/P2_G5_acteurs.csv.gz', compression = 'gzip')
# df_realisateurs = pd.read_csv('BD/P2_G5_realisateurs.csv.gz', compression = 'gzip')

# df_acteurs['knownForTitles'] = df_acteurs['knownForTitles'].astype(str)
# df_acteurs['knownForTitles'] = df_acteurs['knownForTitles'].apply(lambda x : x.replace('[', '').replace(']', '').replace(' ', ''))
# df_acteurs['films'] = df_acteurs['knownForTitles'] + ',' + df_acteurs['film_id_out_KNN']
# df_acteurs['films'] = df_acteurs['films'].apply(lambda x : list(set(x.split(','))))

In [627]:
df_acteurs = df_acteurs.rename({'knownForTitles' : 'films'}, axis = 1)

In [628]:
df_acteurs2 = df_acteurs.explode('films')

In [629]:
acteurs_repetitions = pd.DataFrame(df_acteurs2['acteur'].value_counts()).reset_index()

In [630]:
acteurs_repetitions = acteurs_repetitions[acteurs_repetitions['count'] > apparition_acteur]

In [631]:
acteurs_recurrents = pd.merge(acteurs_repetitions,
                              df_acteurs,
                              how = 'left',
                              on = 'acteur')

In [632]:
acteurs_recurrents2 = acteurs_recurrents.explode('films')

In [633]:
acteurs_recurrents2['films'] = acteurs_recurrents2['films'].str.replace("'","")
acteurs_recurrents2['acteur'] = acteurs_recurrents2['acteur'] + ', '

In [634]:
acteurs_recurrents2 = acteurs_recurrents2.groupby('films').sum()

acteurs_recurrents2['acteur'] = acteurs_recurrents2['acteur'].astype(str)

In [635]:
acteurs_recurrents2['acteur'] = acteurs_recurrents2['acteur'].apply(lambda x : x.replace(", , ", ''))

acteurs_recurrents2 = acteurs_recurrents2.reset_index()
acteurs_recurrents2 = acteurs_recurrents2[['films', 'acteur']]

In [636]:
df_films = pd.merge(df_films,
                    acteurs_recurrents2,
                    how = 'left',
                    left_on = 'film_id',
                    right_on = 'films')

In [637]:
acteurs_repetitions['acteur'] = acteurs_repetitions['acteur'].astype(str)

In [638]:
# DF REALISATEURS

realisateurs_repetitions = pd.DataFrame(df_realisateurs['realisateurs'].value_counts()).reset_index()

realisateurs_repetitions = realisateurs_repetitions[realisateurs_repetitions['count'] > apparition_realisateur]

realisateurs_repetitions2 = realisateurs_repetitions.groupby('realisateurs').sum()

realisateurs_repetitions2 = realisateurs_repetitions2.reset_index()

In [639]:
realisateurs_connus = pd.merge(realisateurs_repetitions2,
                               df_realisateurs,
                               on = 'realisateurs',
                               how = 'left')

In [640]:
realisateurs_connus = realisateurs_connus[['knownForTitles', 'realisateurs']]

realisateurs_connus['realisateurs'] = realisateurs_connus['realisateurs'] + ','

realisateurs_connus2 = realisateurs_connus.groupby('knownForTitles').sum()

In [641]:
df_films = pd.merge(df_films,
                    realisateurs_connus2,
                    how = 'left',
                    left_on = 'film_id',
                    right_on = 'knownForTitles')

df_realisateurs['realisateurs'] = df_realisateurs['realisateurs'].astype(str)

df_films['realisateurs'] = df_films['realisateurs'].astype(str)

In [642]:
df_films['title_debut'] = df_films['title'].apply(debut)
df_films['title_debut'] = df_films['title_debut'].astype(str)

# df['title_len'] = df['title'].apply(lambda row : len(row.replace(' ','')))

SUPPRESSION DES TITLE SANS CYRILLIQUE

In [643]:
df_films = df_films[~df_films['title'].str.contains(r'[^\x00-\x7F]+', regex=True)]

CREATION COLONNES EN BOOL

In [647]:
bool_colonnes(df_films, 'genre')

In [649]:
bool_colonnes(df_films, 'realisateurs')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/1875185821.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/1875185821.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/1875185821.py:64: PerformanceWarning: DataFram

In [650]:
bool_colonnes(df_films, 'acteur')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/1875185821.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}2'] = df_films[colonne].apply(lambda x: x.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(" ", '').lower())
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/1875185821.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjck

In [654]:
bool_colonnes(df_films, 'production_companies_name')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2739/1875185821.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}2'] = df_films[colonne].apply(lambda x: x.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(" ", '').lower())


KeyboardInterrupt: 

In [161]:
# bool_colonnes(df_films, 'title_debut')

DROP DES COLONNES

In [162]:
colonnes = ['production_companies_name', 'genre', 'realisateurs_nan', 'acteur_nan', 'genre2', 'acteur2', 'realisateurs2', 'production_companies_name2', 'acteur_,', 'original_language', 'spoken_languages', 'language', 'original_language_x_list', 'language_x_list', 'films', 'genres_x_list', 'genres_x', 'genres_y', 'original_title', 'title_y', 'title_x', 'primaryTitle', 'originalTitle', 'isAdult', 'runtime', 'release_date', 'vote_average', 'numVotes', 'vote_count', 'startYear', 'endYear', 'runtimeMinutes', 'averageRating', 'difference_runtime', 'difference_vote', 'isOriginalTitle', 'revenue', 'budget', 'ordering', 'types', 'attributes', 'video', 'id', 'films', 'imdb_id', 'tconst', 'titleId', 'production_companies_country', 'acteur', 'realisateurs', 'realisateurs_a', 'realisateurs_7', 'realisateurs_0', 'realisateurs_4', 'realisateurs_9','realisateurs_5', 'realisateurs_8', 'realisateurs_m','realisateurs_3','realisateurs_1','realisateurs_,', 'realisateurs_n','realisateurs_6','realisateurs_2','acteur_a','acteur_7', 'acteur_0','acteur_9','acteur_5','acteur_8', 'acteur_m','acteur_4','acteur_3', 'acteur_1', 'acteur_', 'acteur_ ', 'acteur_n', 'acteur_6', 'acteur_2']

for element in colonnes:
    if element in df_films.columns:
        df_films = df_films.drop(element, axis = 1)

BOOL DES COLONNES

In [163]:
df_films = transfo_col_bool(df_films)

In [164]:
colonnes = list(df_films.select_dtypes(include = 'number').columns)
supprimer = []

df_null = pd.DataFrame(df_films[colonnes].sum()).transpose()

for element in colonnes:
    if df_null[element].iloc[0] == 0:
        supprimer.append(element)

if len(supprimer) > 0:
    df_films = df_films.drop(supprimer, axis = 1)

RENAME DES COLONNES

In [165]:
list_rename = ['production_countries', 'language_fr', 'title_debut', 'titleType', 'title_len', 'region', 'backdrop_path', 'homepage', 'overview', 'poster_path', 'film_id', 'title', 'final_language', 'tagline', 'title']

In [166]:
dico_rename = {}
for element in list_rename:
    if element in df_films.columns:
        dico_rename.update({element: f'{element}_out_KNN'})

In [167]:
df_films = df_films.rename(dico_rename, axis = 1)

In [ ]:
df_films.info()

In [ ]:
df_films.columns

EXPORT

In [168]:
df_films.to_csv('P2_G5_films.csv.gz', index=False, compression='gzip')